# Classification Binaire - Dataset Bancaire
## Playground Series S5E8 - Compétition Kaggle

Ce notebook présente une solution complète pour un problème de classification binaire sur un dataset bancaire. L'objectif est de prédire si un client va souscrire à un produit bancaire (probablement un dépôt à terme).

### Objectifs:
- Explorer et analyser le dataset bancaire
- Effectuer le preprocessing et feature engineering
- Entraîner plusieurs modèles de classification
- Évaluer et comparer les performances
- Générer les prédictions pour la soumission Kaggle

## 1. Import Required Libraries
Importation des bibliothèques nécessaires pour l'analyse de données et l'apprentissage automatique.

In [ ]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Bibliothèques pour le machine learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                            f1_score, roc_auc_score, classification_report, 
                            confusion_matrix, roc_curve)

# Configuration pour les graphiques
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("Toutes les bibliothèques ont été importées avec succès!")

## 2. Load and Explore the Bank Dataset
Chargement et exploration initiale du dataset bancaire pour comprendre la structure des données.

In [ ]:
# Chargement des datasets
train_df = pd.read_csv('playground-series-s5e8/train.csv')
test_df = pd.read_csv('playground-series-s5e8/test.csv')
sample_submission = pd.read_csv('playground-series-s5e8/sample_submission.csv')

print("Datasets chargés avec succès!")
print(f"Train dataset shape: {train_df.shape}")
print(f"Test dataset shape: {test_df.shape}")
print(f"Sample submission shape: {sample_submission.shape}")

# Aperçu du dataset d'entraînement
print("\n=== Aperçu du dataset d'entraînement ===")
train_df.head()

In [ ]:
# Informations générales sur le dataset
print("=== Informations sur le dataset ===")
print("\nInformation sur les colonnes:")
train_df.info()

print("\n=== Distribution de la variable cible ===")
target_distribution = train_df['y'].value_counts()
print(target_distribution)
print(f"\nPourcentage de classe positive (y=1): {train_df['y'].mean():.2%}")

print("\n=== Valeurs manquantes ===")
missing_values = train_df.isnull().sum()
print(missing_values[missing_values > 0])
if missing_values.sum() == 0:
    print("Aucune valeur manquante détectée!")

print("\n=== Statistiques descriptives pour les variables numériques ===")
numeric_cols = train_df.select_dtypes(include=[np.number]).columns
train_df[numeric_cols].describe()

In [ ]:
# Visualisation de la distribution des données
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Distribution de la variable cible
target_counts = train_df['y'].value_counts()
axes[0, 0].bar(target_counts.index, target_counts.values, color=['skyblue', 'lightcoral'])
axes[0, 0].set_title('Distribution de la Variable Cible')
axes[0, 0].set_xlabel('y')
axes[0, 0].set_ylabel('Count')

# Distribution de l'âge
axes[0, 1].hist(train_df['age'], bins=30, alpha=0.7, color='lightgreen')
axes[0, 1].set_title('Distribution de l\'Âge')
axes[0, 1].set_xlabel('Age')
axes[0, 1].set_ylabel('Frequency')

# Balance par classe
train_df.boxplot(column='balance', by='y', ax=axes[0, 2])
axes[0, 2].set_title('Balance par Classe')
axes[0, 2].set_xlabel('y')

# Top 10 jobs
job_counts = train_df['job'].value_counts().head(10)
axes[1, 0].bar(range(len(job_counts)), job_counts.values, color='lightblue')
axes[1, 0].set_title('Top 10 Professions')
axes[1, 0].set_xticks(range(len(job_counts)))
axes[1, 0].set_xticklabels(job_counts.index, rotation=45)

# Education vs target
education_target = pd.crosstab(train_df['education'], train_df['y'], normalize='index')
education_target.plot(kind='bar', stacked=True, ax=axes[1, 1], color=['skyblue', 'lightcoral'])
axes[1, 1].set_title('Education vs Target')
axes[1, 1].set_xlabel('Education')
axes[1, 1].legend(['y=0', 'y=1'])

# Duration distribution
axes[1, 2].hist(train_df['duration'], bins=50, alpha=0.7, color='orange')
axes[1, 2].set_title('Distribution de la Durée')
axes[1, 2].set_xlabel('Duration (seconds)')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 3. Data Preprocessing and Feature Engineering
Préparation des données avec encodage des variables catégorielles, normalisation et création de nouvelles features.

In [ ]:
# Séparation des features et de la target
X = train_df.drop(['id', 'y'], axis=1).copy()
y = train_df['y'].copy()
X_test = test_df.drop(['id'], axis=1).copy()

print("Données séparées:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X_test shape: {X_test.shape}")

# Identification des colonnes catégorielles et numériques
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()

print(f"\nColonnes catégorielles ({len(categorical_cols)}): {categorical_cols}")
print(f"Colonnes numériques ({len(numerical_cols)}): {numerical_cols}")

# Encodage des variables catégorielles
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))
    label_encoders[col] = le
    print(f"Encodé {col}: {le.classes_[:5]}..." if len(le.classes_) > 5 else f"Encodé {col}: {le.classes_}")

print("\nEncodage des variables catégorielles terminé!")

In [ ]:
# Feature Engineering - Création de nouvelles features
def create_features(df):
    df = df.copy()
    
    # Ratio balance/age
    df['balance_age_ratio'] = df['balance'] / (df['age'] + 1)
    
    # Indicateur de balance positive/négative
    df['balance_positive'] = (df['balance'] > 0).astype(int)
    
    # Durée en minutes
    df['duration_minutes'] = df['duration'] / 60
    
    # Combinaison housing + loan
    df['housing_loan_sum'] = df['housing'] + df['loan']
    
    # Age en catégories
    df['age_group'] = pd.cut(df['age'], bins=[0, 25, 35, 50, 100], labels=[0, 1, 2, 3])
    df['age_group'] = df['age_group'].astype(int)
    
    # Interaction entre campaign et previous
    df['campaign_previous_interaction'] = df['campaign'] * (df['previous'] + 1)
    
    return df

# Appliquer le feature engineering
X_engineered = create_features(X)
X_test_engineered = create_features(X_test)

print("Feature engineering terminé!")
print(f"Nombre de features originales: {X.shape[1]}")
print(f"Nombre de features après engineering: {X_engineered.shape[1]}")
print(f"Nouvelles features créées: {X_engineered.shape[1] - X.shape[1]}")

# Normalisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_engineered)
X_test_scaled = scaler.transform(X_test_engineered)

# Conversion en DataFrame pour garder les noms de colonnes
X_scaled = pd.DataFrame(X_scaled, columns=X_engineered.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_engineered.columns)

print("\nNormalisation terminée!")
print(f"Shape finale: X_scaled {X_scaled.shape}, X_test_scaled {X_test_scaled.shape}")

## 4. Split Data into Training and Testing Sets
Division des données en ensembles d'entraînement et de validation avec stratification pour maintenir la distribution de la variable cible.

In [ ]:
# Division des données en train/validation
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print("Division des données terminée:")
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")

# Vérification de la distribution de la variable cible
print(f"\nDistribution dans l'ensemble d'entraînement:")
print(f"Classe 0: {(y_train == 0).sum()} ({(y_train == 0).mean():.2%})")
print(f"Classe 1: {(y_train == 1).sum()} ({(y_train == 1).mean():.2%})")

print(f"\nDistribution dans l'ensemble de validation:")
print(f"Classe 0: {(y_val == 0).sum()} ({(y_val == 0).mean():.2%})")
print(f"Classe 1: {(y_val == 1).sum()} ({(y_val == 1).mean():.2%})")

## 5. Train Binary Classification Models
Entraînement de plusieurs algorithmes de classification binaire incluant Logistic Regression, Random Forest et SVM.

In [ ]:
# Définition des modèles à entraîner
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, max_depth=6, random_state=42),
    'SVM': SVC(random_state=42, probability=True)
}

# Entraînement des modèles
trained_models = {}
training_results = {}

print("Entraînement des modèles en cours...\n")

for name, model in models.items():
    print(f"Entraînement de {name}...")
    
    # Entraînement
    model.fit(X_train, y_train)
    trained_models[name] = model
    
    # Prédictions sur l'ensemble de validation
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    
    # Calcul des métriques
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    auc = roc_auc_score(y_val, y_pred_proba)
    
    training_results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  AUC: {auc:.4f}")
    print("-" * 50)

print("Entraînement de tous les modèles terminé!")

## 6. Model Evaluation and Performance Metrics
Évaluation détaillée des performances des modèles avec métriques complètes et visualisations.

In [ ]:
# Création d'un tableau de comparaison des modèles
results_df = pd.DataFrame(training_results).T
print("=== Comparaison des Performances des Modèles ===")
print(results_df[['accuracy', 'precision', 'recall', 'f1', 'auc']].round(4))

# Visualisation des performances
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Graphique en barres des métriques
metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc']
x = np.arange(len(metrics))
width = 0.2

for i, (model_name, model_results) in enumerate(training_results.items()):
    values = [model_results[metric] for metric in metrics]
    axes[0, 0].bar(x + i*width, values, width, label=model_name)

axes[0, 0].set_xlabel('Métriques')
axes[0, 0].set_ylabel('Score')
axes[0, 0].set_title('Comparaison des Métriques par Modèle')
axes[0, 0].set_xticks(x + width * 1.5)
axes[0, 0].set_xticklabels(metrics)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Courbes ROC
for model_name, model_results in training_results.items():
    fpr, tpr, _ = roc_curve(y_val, model_results['probabilities'])
    auc_score = model_results['auc']
    axes[0, 1].plot(fpr, tpr, label=f'{model_name} (AUC = {auc_score:.3f})')

axes[0, 1].plot([0, 1], [0, 1], 'k--', label='Random')
axes[0, 1].set_xlabel('Taux de Faux Positifs')
axes[0, 1].set_ylabel('Taux de Vrais Positifs')
axes[0, 1].set_title('Courbes ROC')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Matrices de confusion pour les 2 meilleurs modèles
best_models = results_df.nlargest(2, 'auc').index

for i, model_name in enumerate(best_models):
    cm = confusion_matrix(y_val, training_results[model_name]['predictions'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, i])
    axes[1, i].set_title(f'Matrice de Confusion - {model_name}')
    axes[1, i].set_xlabel('Prédictions')
    axes[1, i].set_ylabel('Valeurs Réelles')

plt.tight_layout()
plt.show()

# Identifier le meilleur modèle
best_model_name = results_df['auc'].idxmax()
print(f"\n🏆 Meilleur modèle: {best_model_name} (AUC: {results_df.loc[best_model_name, 'auc']:.4f})")

## 7. Feature Importance Analysis
Analyse de l'importance des features pour comprendre quelles variables influencent le plus la prédiction.

In [ ]:
# Analyse de l'importance des features pour les modèles tree-based
tree_models = ['Random Forest', 'Gradient Boosting']

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

for i, model_name in enumerate(tree_models):
    if model_name in trained_models:
        model = trained_models[model_name]
        feature_importance = model.feature_importances_
        feature_names = X_scaled.columns
        
        # Créer un DataFrame pour l'importance des features
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': feature_importance
        }).sort_values('importance', ascending=False)
        
        # Afficher le top 20 des features
        top_features = importance_df.head(20)
        
        axes[i].barh(range(len(top_features)), top_features['importance'])
        axes[i].set_yticks(range(len(top_features)))
        axes[i].set_yticklabels(top_features['feature'])
        axes[i].set_xlabel('Importance')
        axes[i].set_title(f'Top 20 Features - {model_name}')
        axes[i].grid(True, alpha=0.3)
        
        # Inverser l'ordre des features pour un affichage plus intuitif
        axes[i].invert_yaxis()
        
        print(f"\n=== Top 10 Features pour {model_name} ===")
        print(importance_df.head(10).to_string(index=False))

plt.tight_layout()
plt.show()

# Analyse des coefficients pour la Logistic Regression
if 'Logistic Regression' in trained_models:
    lr_model = trained_models['Logistic Regression']
    coefficients = lr_model.coef_[0]
    feature_names = X_scaled.columns
    
    # Créer un DataFrame pour les coefficients
    coef_df = pd.DataFrame({
        'feature': feature_names,
        'coefficient': coefficients,
        'abs_coefficient': np.abs(coefficients)
    }).sort_values('abs_coefficient', ascending=False)
    
    # Visualisation des coefficients
    plt.figure(figsize=(12, 8))
    top_coef = coef_df.head(20)
    colors = ['red' if x < 0 else 'blue' for x in top_coef['coefficient']]
    
    plt.barh(range(len(top_coef)), top_coef['coefficient'], color=colors)
    plt.yticks(range(len(top_coef)), top_coef['feature'])
    plt.xlabel('Coefficient')
    plt.title('Top 20 Coefficients - Logistic Regression')
    plt.grid(True, alpha=0.3)
    plt.gca().invert_yaxis()
    
    # Ajouter une légende
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n=== Top 10 Coefficients (valeur absolue) pour Logistic Regression ===")
    print(coef_df.head(10)[['feature', 'coefficient']].to_string(index=False))

## 8. Model Comparison and Selection
Comparaison finale des modèles avec validation croisée et génération des prédictions pour la soumission Kaggle.

In [ ]:
# Validation croisée pour une évaluation plus robuste
print("=== Validation Croisée (5-fold) ===")
cv_results = {}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in trained_models.items():
    print(f"\nValidation croisée pour {name}...")
    
    # Cross-validation scores
    cv_scores = cross_val_score(model, X_scaled, y, cv=skf, scoring='roc_auc', n_jobs=-1)
    
    cv_results[name] = {
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'cv_scores': cv_scores
    }
    
    print(f"  AUC CV: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    print(f"  Scores: {cv_scores.round(4)}")

# Tableau final de comparaison
final_comparison = pd.DataFrame({
    'Validation_AUC': [training_results[name]['auc'] for name in trained_models.keys()],
    'CV_AUC_Mean': [cv_results[name]['cv_mean'] for name in trained_models.keys()],
    'CV_AUC_Std': [cv_results[name]['cv_std'] for name in trained_models.keys()]
}, index=trained_models.keys())

print("\n=== Comparaison Finale des Modèles ===")
print(final_comparison.round(4))

# Sélection du meilleur modèle basé sur la validation croisée
best_model_cv = final_comparison['CV_AUC_Mean'].idxmax()
best_model = trained_models[best_model_cv]

print(f"\n🏆 Modèle sélectionné: {best_model_cv}")
print(f"   AUC Validation: {final_comparison.loc[best_model_cv, 'Validation_AUC']:.4f}")
print(f"   AUC CV Mean: {final_comparison.loc[best_model_cv, 'CV_AUC_Mean']:.4f}")
print(f"   AUC CV Std: {final_comparison.loc[best_model_cv, 'CV_AUC_Std']:.4f}")

In [ ]:
# Génération des prédictions finales pour la soumission
print("=== Génération des Prédictions Finales ===")

# Entraîner le meilleur modèle sur l'ensemble complet des données d'entraînement
final_model = trained_models[best_model_cv]
final_model.fit(X_scaled, y)

# Prédictions sur le test set
test_predictions_proba = final_model.predict_proba(X_test_scaled)[:, 1]

# Créer le fichier de soumission
submission = pd.DataFrame({
    'id': test_df['id'],
    'y': test_predictions_proba
})

# Sauvegarder le fichier de soumission
submission.to_csv('submission.csv', index=False)

print(f"✅ Fichier de soumission créé: submission.csv")
print(f"   Nombre de prédictions: {len(submission)}")
print(f"   Modèle utilisé: {best_model_cv}")
print(f"   Range des prédictions: [{test_predictions_proba.min():.4f}, {test_predictions_proba.max():.4f}]")
print(f"   Moyenne des prédictions: {test_predictions_proba.mean():.4f}")

# Aperçu du fichier de soumission
print("\n=== Aperçu du fichier de soumission ===")
print(submission.head(10))

# Statistiques des prédictions
print(f"\n=== Statistiques des Prédictions ===")
print(f"Prédictions < 0.1: {(test_predictions_proba < 0.1).sum()} ({(test_predictions_proba < 0.1).mean():.2%})")
print(f"Prédictions 0.1-0.5: {((test_predictions_proba >= 0.1) & (test_predictions_proba < 0.5)).sum()} ({((test_predictions_proba >= 0.1) & (test_predictions_proba < 0.5)).mean():.2%})")
print(f"Prédictions >= 0.5: {(test_predictions_proba >= 0.5).sum()} ({(test_predictions_proba >= 0.5).mean():.2%})")

# Histogramme des prédictions
plt.figure(figsize=(10, 6))
plt.hist(test_predictions_proba, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
plt.xlabel('Probabilité Prédite')
plt.ylabel('Fréquence')
plt.title('Distribution des Probabilités Prédites')
plt.grid(True, alpha=0.3)
plt.axvline(test_predictions_proba.mean(), color='red', linestyle='--', label=f'Moyenne: {test_predictions_proba.mean():.3f}')
plt.legend()
plt.show()

print("\n🎯 Classification binaire terminée avec succès!")